### **Four datas of meat process**

In [2]:
import pandas as pd
data1 = pd.read_csv("Datathon Data/acs_5yr_est_selected_economic_characteristics_2010-2022.csv")
deheader = data1.loc[data1['Category'] != 'Header']
# deheader.head(20)

读取数据

In [3]:
storage_data1 = pd.read_csv('Meat_Stats_Cold_Storage.csv')
production_data2 = pd.read_csv('Meat_Stats_Meat_Production.csv')
counts_data3 = pd.read_csv('Meat_Stats_Slaughter_Counts.csv')
slaughter_data4 = pd.read_csv('Meat_Stats_Slaughter_Weights.csv')
slaughter_data4.head(5)

,Unnamed: 0,Date,Year,Month,Unit,Animal,Weight,Commercial_Or_Federally_Inspected,Live_Or_Dressed,Type_Of_Meat
0,0,Dec-23,2023,12,Average Weight in Pounds,Cattle,1401.0,Commercial,Live,Red Meat
1,1,Nov-23,2023,11,Average Weight in Pounds,Cattle,1387.0,Commercial,Live,Red Meat
2,2,Oct-23,2023,10,Average Weight in Pounds,Cattle,1374.0,Commercial,Live,Red Meat
3,3,Sep-23,2023,9,Average Weight in Pounds,Cattle,1362.0,Commercial,Live,Red Meat
4,4,Aug-23,2023,8,Average Weight in Pounds,Cattle,1350.0,Commercial,Live,Red Meat


统一主数据列名为Amount，方便统一编写函数

In [4]:
def replace_columns(data,name):
    ans = []
    for i in list(data.columns):
        if i == name:
            ans.append('Amount')
        else:
            ans.append(i)
    data.columns = ans
    Counts = []
    for i in list(data['Amount']):
        Counts.append(float(str(i).replace(',','')))
    data['Amount'] = Counts

replace_columns(production_data2,'Weight')
replace_columns(_data3,'Production')
replace_columns(data4,'Count')
replace_columns(data5,'Weight')

data4.head(5)


NameError: name 'data2' is not defined

时间序列折线图绘制函数

In [ ]:
import matplotlib.pyplot as plt
import datetime as dt

def plot_strange(data):
    uniq = data['Animal'].unique()
    n = len(uniq)
    
    fig = plt.figure(figsize=(15,2.5*n))
    
    for i in range(n):
        datanew = data.loc[data['Animal'] == uniq[i]]
        x = []
        for k in range(datanew.shape[0]):
            strk = '{}-{}'.format(list(datanew['Year'])[k],list(datanew['Month'])[k])
            x.append(dt.datetime.strptime(strk,'%Y-%m'))
        
        y = datanew['Amount']
        ax = fig.add_subplot(n//2+1,2, i+1)
        # print(len(x),len(y),x[0])
        # x=[i for i in range(2024)], [1]*2024
        ax.plot(x,y)
        ax.set_title(data['Animal'].unique()[i])
    plt.show()

plot_strange(data2)


箱线图绘制函数

In [ ]:
def animal_boxplot(data):
    uniq = data['Animal'].unique()
    n = len(uniq)
    fig = plt.figure(figsize=(12,2*n))
    for i in range(n):
        datanew = data.loc[data['Animal'] == uniq[i]]
        y = datanew['Amount']
        # print(uniq[i],y.describe())
        ax = fig.add_subplot(n//3+1,3, i+1)
        ax.boxplot(y.dropna())
        ax.set_title(uniq[i])
    plt.show()


绘制data4的箱线图

In [ ]:
animal_boxplot(data3)

发现有异常值，去除异常值。

In [ ]:
def drop_strange(data):
    uniq = data['Animal'].unique()
    n = len(uniq)
    # fig = plt.figure(figsize=(12,2*n))
    dic_data = dict()
    for i in range(n):
        datanew = data.loc[data['Animal'] == uniq[i]]
        y = datanew['Amount']
        IQR = y.quantile(0.75) - y.quantile(0.25)
        dic_data[uniq[i]] = (y.median()-2.5*IQR , y.median()+2.5*IQR) # 波动区间
    for k in range(data.shape[0]):
        if data.loc[k,'Animal'] == 'Turkeys':
            continue
        intercept = dic_data[data['Animal'][k]]
        if (data.loc[k,'Amount']< intercept[0] or data.loc[k,'Amount'] > intercept[1]):
            data.loc[k,'Amount'] = data.loc[k-1,'Amount'] if data.loc[k-1,'Animal']== data.loc[k,'Animal'] else data.loc[k+1,'Amount']

drop_strange(data2)
drop_strange(data3)
drop_strange(data4)
drop_strange(data5)


In [ ]:
data3.describe()

再次绘制data4的箱线图查看情况:

In [ ]:
animal_boxplot(data4)

↑正常了好多

保存csv：


In [ ]:
# data2.to_csv(path_or_buf = "C:/Users/Zhang/Desktop/python/Citadel/Datathon Data/Meat_Stats_Cold_Storage.csv")
data3.to_csv(path_or_buf = "C:/Users/Zhang/Desktop/python/Citadel/Datathon Data/Meat_Stats_Meat_Production.csv")
# data4.to_csv(path_or_buf = "C:/Users/Zhang/Desktop/python/Citadel/Datathon Data/Meat_Stats_Slaughter_Counts.csv")
# data5.to_csv(path_or_buf = "C:/Users/Zhang/Desktop/python/Citadel/Datathon Data/Meat_Stats_Slaughter_Weights.csv")